In [1]:
import json

In [2]:
def get_str_map(str_list):
    str2id = {}
    id2str = {}
    for i, tag in enumerate(str_list):
        str2id[tag] = i
        id2str[i] = tag
    return id2str, str2id

def read_data(path):
    symp_set = set()
    disease_set = set()
    statistics = {}
    with open('diagnose_train.json', 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for line in lines:
            line = json.loads(line)
            symptoms = line['symptoms']
            disease = line['disease']
            disease_set.add(disease)
            if disease not in statistics:
                statistics[disease] = {'symptoms':set(), 'sample_cnt':0}

            for symp in symptoms:
                symp_set.add(symp)
                statistics[disease]['symptoms'].add(symp)
            statistics[disease]['sample_cnt'] += 1

    return symp_set, disease_set, statistics


In [3]:
symp_set, disease_set, statistics= read_data('diagnose_train.json')
id2str_symp, str2id_symp = get_str_map(list(symp_set))
id2str_disease, str2id_disease = get_str_map(list(disease_set))

for dise, attrs in statistics.items():
    print(dise + ": " + "; ".join("{}: {}".format(k, v if type(v) is int else len(v)) for k, v in attrs.items()))

print(f'total disease: {len(disease_set)}; total symtoms:{len(symp_set)}; total sample:{sum([v if type(v) is int else 0 for (dise, attrs) in statistics.items() for (k,v) in attrs.items()])}')

with open('id2symptom.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(id2str_symp, indent=4, ensure_ascii=False))

with open('id2disease.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(id2str_disease, indent=4, ensure_ascii=False))

with open('symptom2id.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(str2id_symp, indent=4, ensure_ascii=False))

with open('disease2id.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(str2id_disease, indent=4, ensure_ascii=False))


上呼吸道感染: symptoms: 58; sample_cnt: 130
小儿支气管炎: symptoms: 60; sample_cnt: 166
小儿腹泻: symptoms: 55; sample_cnt: 155
小儿消化不良: symptoms: 56; sample_cnt: 117
total disease: 4; total symtoms:66; total sample:568


# K-Fold Crossvalidation

In [4]:
from sklearn.model_selection import KFold
SPLIT_NUM = 10
kf = KFold(n_splits=SPLIT_NUM, random_state=2021, shuffle=True)

cnt = 0
with open('diagnose_train.json', 'r', encoding='utf-8') as f:
    all_lines = f.readlines()
    for train_index, valid_index in kf.split(all_lines):
        with open(f'K-fold/diag_rule_fold_{cnt}.json', 'w', encoding='utf-8') as f_rule, open(f'K-fold/diag_valid_fold_{cnt}.json', 'w', encoding='utf-8') as f_data:
            f_rule.writelines([all_lines[i] for i in train_index])
            f_data.writelines([all_lines[i] for i in valid_index])
            cnt += 1

# Incremental Setting

In [5]:
import json
with open('diagnose.json', 'r', encoding='utf-8') as f_diag, open('disease2id.json', 'r', encoding='utf-8') as f_dise:
    all_lines = f_diag.readlines()
    diseases = json.loads(f_dise.read())
    for dise in diseases:
        with open(f'IncreSetting/diagnose_inven_{dise}.json', 'w', encoding='utf-8') as f_inven, open(f'IncreSetting/diagnose_incre_{dise}.json', 'w', encoding='utf-8') as f_incre:
            for line in all_lines:
                disease = json.loads(line)['disease']
                f_inven.write(line) if disease != dise else f_incre.write(line)

FileNotFoundError: [Errno 2] No such file or directory: 'diagnose.json'